In [5]:
import pandas as pd
import numpy as np

# === Step 1: Load Dataset ===
df = pd.read_csv("pbp-2023.csv")

# === Step 2: Clean Dataset ===
df = df.dropna(axis=1, how='all')
df = df.drop_duplicates()

# ----------------------------------------------------------------------
# ORIGINAL 6 QUESTIONS
# ----------------------------------------------------------------------

# 1. Total games
total_games = df['GameId'].nunique()

# 2. Team with most touchdowns
touchdowns_per_team = (
    df[df['IsTouchdown'] == 1]
    .groupby('OffenseTeam')
    .size()
    .sort_values(ascending=False)
)
most_touchdowns_team = touchdowns_per_team.idxmax()
most_touchdowns_count = touchdowns_per_team.max()

# 3. Average yards per play
avg_yards_per_play = df['Yards'].mean()

# 4. Team with most passing touchdowns
passing_tds_per_team = (
    df[(df['PlayType'] == "PASS") & (df['IsTouchdown'] == 1)]
    .groupby('OffenseTeam')
    .size()
    .sort_values(ascending=False)
)
most_passing_tds_team = passing_tds_per_team.idxmax()
most_passing_tds_count = passing_tds_per_team.max()

# 5. Offensive vs. defensive focus metric
avg_yards_pass = df[df['IsPass'] == 1]['Yards'].mean()
avg_yards_rush = df[df['IsRush'] == 1]['Yards'].mean()

# 6. Most impactful player (TDs + yards)
impact_data = (
    df[df['IsTouchdown'] == 1]
    .groupby('Description')['Yards']
    .agg(['count', 'sum'])
    .rename(columns={'count': 'TDs', 'sum': 'TotalYards'})
)
impact_data['ImpactScore'] = impact_data['TDs'] + (impact_data['TotalYards'] / 10)
impactful_player = impact_data.sort_values('ImpactScore', ascending=False).head(1)

# Save original summary
original_summary = {
    "total_games": total_games,
    "most_touchdowns_team": most_touchdowns_team,
    "most_touchdowns_count": int(most_touchdowns_count),
    "avg_yards_per_play": avg_yards_per_play,
    "most_passing_tds_team": most_passing_tds_team,
    "most_passing_tds_count": int(most_passing_tds_count),
    "avg_yards_pass": avg_yards_pass,
    "avg_yards_rush": avg_yards_rush,
    "most_impactful_player": impactful_player.to_dict()
}
pd.Series(original_summary).to_csv("summary_stats.csv")
print("Original summary stats saved to 'summary_stats.csv'")

# ----------------------------------------------------------------------
# ADVANCED 10 QUESTIONS
# ----------------------------------------------------------------------

# Add season half info
df['GameDate'] = pd.to_datetime(df['GameDate'])
games_sorted = df['GameId'].drop_duplicates().sort_values().tolist()
half_split = len(games_sorted) // 2
first_half_games = games_sorted[:half_split]
df['SeasonHalf'] = df['GameId'].apply(lambda x: 'First' if x in first_half_games else 'Second')

# 1. Highest avg yards/play in wins
avg_yards_win = (
    df[df['TeamWin'] == 1]
    .groupby('OffenseTeam')['Yards']
    .mean()
    .sort_values(ascending=False)
)

# 2. Defense allowed fewest passing TDs
pass_tds_allowed = (
    df[(df['PlayType'] == "PASS") & (df['IsTouchdown'] == 1)]
    .groupby('DefenseTeam')
    .size()
    .sort_values()
)

# 3. Quarter with highest scoring rate
quarter_scoring = (
    df[df['IsTouchdown'] == 1]
    .groupby('Quarter')
    .size()
    / df.groupby('Quarter').size()
).sort_values(ascending=False)

# 4. Most improved team from first to second half
team_half_avg = df.groupby(['OffenseTeam', 'SeasonHalf'])['Yards'].mean().unstack()
team_half_avg['Improvement'] = team_half_avg['Second'] - team_half_avg['First']
most_improved_team = team_half_avg['Improvement'].idxmax()

# 5. Player with largest % of team touchdowns
tds_by_player_team = (
    df[df['IsTouchdown'] == 1]
    .groupby(['OffenseTeam', 'Description'])
    .size()
    .reset_index(name='PlayerTDs')
)
tds_by_team = (
    df[df['IsTouchdown'] == 1]
    .groupby('OffenseTeam')
    .size()
    .reset_index(name='TeamTDs')
)
player_td_pct = pd.merge(tds_by_player_team, tds_by_team, on='OffenseTeam')
player_td_pct['Pct'] = player_td_pct['PlayerTDs'] / player_td_pct['TeamTDs']
top_player_pct = player_td_pct.sort_values('Pct', ascending=False).head(1)

# 6. Rush vs pass defense recommendation
avg_yards_pass_allowed = df[df['IsPass'] == 1]['Yards'].mean()
avg_yards_rush_allowed = df[df['IsRush'] == 1]['Yards'].mean()

# 7. Most efficient red-zone offense
red_zone = df[df['YardLineFixed'] <= 20]
red_zone_eff = (
    red_zone.groupby('OffenseTeam').apply(lambda x: (x['IsTouchdown'].sum() / len(x)))
).sort_values(ascending=False)

# 8. DAL passing TDs +10%
dal_pass_tds = len(df[(df['OffenseTeam'] == 'DAL') & (df['PlayType'] == "PASS") & (df['IsTouchdown'] == 1)])
dal_pass_tds_10pct = dal_pass_tds * 1.10

# 9. Most competitive matchup
avg_yards_team = df.groupby('OffenseTeam')['Yards'].mean()
matchup_diff = {}
teams = avg_yards_team.index.tolist()
for i in range(len(teams)):
    for j in range(i+1, len(teams)):
        diff = abs(avg_yards_team[teams[i]] - avg_yards_team[teams[j]])
        matchup_diff[(teams[i], teams[j])] = diff
closest_matchup = min(matchup_diff, key=matchup_diff.get)

# 10. Best balanced team (rush vs pass)
rush_avg_team = df[df['IsRush'] == 1].groupby('OffenseTeam')['Yards'].mean()
pass_avg_team = df[df['IsPass'] == 1].groupby('OffenseTeam')['Yards'].mean()
balance_score = abs(rush_avg_team - pass_avg_team)
best_balanced_team = balance_score.idxmin()

# Save advanced summary
advanced_summary = {
    "highest_avg_yards_per_play_in_wins": avg_yards_win.head(1).to_dict(),
    "defense_fewest_passing_TDs": pass_tds_allowed.head(1).to_dict(),
    "quarter_highest_scoring_rate": quarter_scoring.head(1).to_dict(),
    "most_improved_team_half_to_half": most_improved_team,
    "top_player_pct_team_TDs": top_player_pct.to_dict(orient="records"),
    "defensive_focus": {
        "pass_yards_allowed": avg_yards_pass_allowed,
        "rush_yards_allowed": avg_yards_rush_allowed
    },
    "most_efficient_red_zone_offense": red_zone_eff.head(1).to_dict(),
    "DAL_pass_TDs_plus_10pct": dal_pass_tds_10pct,
    "most_competitive_matchup": closest_matchup,
    "best_balanced_team": best_balanced_team
}
pd.Series(advanced_summary).to_csv("advanced_summary_stats.csv")
print("Advanced summary stats saved to 'advanced_summary_stats.csv'")

# ----------------------------------------------------------------------
# OPTIONAL: Merge original + advanced into one CSV
# ----------------------------------------------------------------------
combined_summary = {**original_summary, **advanced_summary}
pd.Series(combined_summary).to_csv("combined_summary_stats.csv")
print("Combined summary stats saved to 'combined_summary_stats.csv'")


Original summary stats saved to 'summary_stats.csv'
Advanced summary stats saved to 'advanced_summary_stats.csv'
Combined summary stats saved to 'combined_summary_stats.csv'


/tmp/ipython-input-4250219956.py:134: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  red_zone.groupby('OffenseTeam').apply(lambda x: (x['IsTouchdown'].sum() / len(x)))
